## 더 나은 성능의 tf.function
##### url: https://www.tensorflow.org/tutorials/customization/performance

- 목차
    - 준비
    - 기초
    - 추적과 다형성
    - 재추적 시기
    - 파이썬 또는 텐서 인자
    - tf.function의 부작용
    - 파이썬 상태 값의 주의점
    - 자동 제어 의존성
    - 변수
    - AutoGraph 사용하기
    - AutoGraph: 조건
    - AutoGraph와 반복문
        - 루프
        - 찾은 것
    - 더 읽어볼 내용
    
Tensorflow 2.0에서는 즉시 실행 기능을 기본으로 채택하고 있습니다. 유저 인터페이스는 직관적이고 유연(하나의 연산을 실행하는것이 더욱 쉽고 빠릅니다.)하지만, 배포성과 성능 측면에서 비용을 지불하고 있습니다.<br><br>
최대 성능을 얻는 것과 어디에서든 배포가 가능한 모델을 만들기 위해서는 `tf.function`을 사용하여 프로그램의 그래프를 만들어야합니다. 고맙게도 AutoGraph는, 놀라운 양의 파이썬 코드로 tf.function과 함께 작동하지만 여전히 주의해야할 어려움이 존재합니다.<br><br>
다음은 피해야하는 것과 추천되는 것들 입니다.
- 변환 가능한 객체와 리스트 어펜드 같은 파이썬 부작용에 의존하지마라 
- tf.function은 Tensorflow 연산을 NumPy나 원시 파이썬 연산보다 가장 잘(best) 작동시킨다.
- `for x in y` 같은 관용구를 사용할 때 의심해라.

## 준비

In [1]:
import tensorflow as tf

직면할 수 있는 에러들을 시연하기 위해 helper 함수를 정의하겠습니다.

In [13]:
import traceback
import contextlib

# 직면할 수 있는 에러를 시연하기 위한 몇몇의 helper 코드
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('예상되는 예외를 찾았습니다. \n {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('예상된 {}가 발생하였으나 에러는 발생하지 않았습니다.'.format(error_class))

## 기초

당신이 정의한 `tf.function`은 핵심 Tensorflow 연산과 동일합니다.: 즉시 실행으로 사용할 수 있습니다; 그래프 내에서 사용할 수 있습니다; 그라디언트를 가집니다; 

In [3]:
@tf.function
def add(a, b):
    return a + b

add(tf.ones([2, 2]), tf.ones([2, 2])) # [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [5]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

함수 내에서 함수를 사용할 수도 있습니다.

In [6]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

## Trace와 다형성

파이썬의 동적 타이핑은 다양한 인자 타입으로 함수를 호출할 수 있다는 것을 의미합니다. 그리고 파이썬은 각 시나리오에 대해 몇몇은 다르게 작동합니다.<br><br>
반면에, Tensorflow 그래프는 정적인 타입과 차원을 요구합니다. `tf.function`은 올바른 그래프를 생성을 필요로 할 때 함수를 Retrace하여 이러한 차이를 연결해줍니다. <br>
대부분의 `tf.function`에 대한 미묘함은 Retrace 동작에서 비롯됩니다.<br><br>
서로 다른 타입의 인자를 전달하여 어떤 일이 일어나는지 함수를 호출해볼 수 있습니다.

In [9]:
# 함수는 다양한 형을 가집니다.

@tf.function
def double(a):
    print('Tracing with', a)
    return a + a

print(double(tf.constant(1)), end='\n\n')
print(double(tf.constant(1.1)), end='\n\n')
print(double(tf.constant('a')), end='\n\n')

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



트레이스 동작을 제어하기위해, 다음과 같은 테크닉을 사용합니다.<br>

- 새로운 tf.function 생성하기. Trace를 공유하지 않도록 tf.function 를 분리하여 보장합니다.
- 특정한 Trace를 얻기 위해 `get_concrete_funciton` 메소드를 사용합니다.
- 그래프를 호출할 때마다 한번만 Trace 하도록 `tf.function`을 호출할 때 `input_signature`를 명확화 합니다.

In [11]:
print('구체적인 Trace 얻기')
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None,
                                                            dtype=tf.string))
print('Traced 함수 수행하기')
print(double_strings(tf.constant('a')))
print(double_strings(a=tf.constant('b')))
print('호환이 되지 않는 타입으로 특정 trace를 사용하는 것은 오류를 발생시킵니다.')
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

구체적인 Trace 얻기
Traced 함수 수행하기
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
호환이 되지 않는 타입으로 특정 trace를 사용하는 것은 오류를 발생시킵니다.
예상되던 예외를 찾았습니다. 
 <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-2-c00eb3beca2a>", line 8, in assert_raises
    yield
  File "<ipython-input-11-97fe1bcb1b7c>", line 9, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_117 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_117]


In [14]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print('Tracing with', x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# 입력 시그니처에서 1 차원 텐서로 지정했습니다, 따라서 다음 구문은 실패하게 됩니다.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
예상되는 예외를 찾았습니다. 
 <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-13-4dbdae590238>", line 8, in assert_raises
    yield
  File "<ipython-input-14-1e97437e7e0d>", line 9, in <module>
    next_collatz(tf.constant([[1, 2], [3, 4]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))
